In [100]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Hide Code"></form>''')

In [101]:
###Brand Pulse Dashboard
#Import Necessary Packages
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import HTML
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
from sklearn.linear_model import LinearRegression

In [102]:
## Loading Data
PastYearData =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/PastYearDataNew.csv", encoding='utf-8')	
CurrentYearData =pd.read_csv("//192.168.2.32/Group/Data Team/Brand_Update_Location/11_Brand_Performance_Analysis/CurrentYearDataNew.csv", encoding='utf-8')	

#Combine Past and recent years sales data
SD = pd.concat([PastYearData, CurrentYearData], ignore_index=True)

Y:\Abul\Conda\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (2,16,17,34,35,36,40,41,44,49,50,51,52,58) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [104]:
# Convert Order_Date string to date and extract relevant date values
SD['Order_Date'] = pd.to_datetime(SD['Order_Date'], format= "%d-%b-%y")
SD['OD_Year'] = SD['Order_Date'].dt.strftime('%Y')
SD['OD_MonthNum'] = SD['Order_Date'].dt.strftime('%m')
SD['OD_MonthLab'] = SD['Order_Date'].dt.strftime('%B')
SD['OD_MonthDay'] = SD['Order_Date'].dt.strftime('%d')
SD['OD_WeekDay'] = SD['Order_Date'].dt.strftime('%A')


In [105]:
#Extract meta data from SD
##Identify all unique brands
AllBrands = SD.attribute_set.unique()
AllSuppliers = SD.Supplier.unique()
AllBucketValues = SD.Order_Bucket.unique()

#list(SD.columns) 

In [106]:
# Filter out any blank orderdate values
FilterSD =    SD[(SD['Order_Date']!="")
           #& (SD['Supplier']=="")
           ]

    #Setup Interactive Global Filter
###Reconciliation_Status <-Set up check box filter
###Cost_Recording_Status <-Set up check box filter
###Order_Bucket>"Complete Bucket" <-Set up check box filter
###Order_Bucket>"Pending Bucket" <-Set up check box filter

In [107]:
#Group by Attribute set & year sold, then count number of sales
GroupedSD = (FilterSD.groupby(["attribute_set","OD_Year"], as_index=False)
                ['Order_Date'].
                agg({"count":"count"}).sort_values(["attribute_set","OD_Year"] , ascending = True))


In [108]:
#Compiled Output functions
#Plot Brands sales per year 
def BrandPlotter(SB):
    
    
#Setup Interactive Global Filter
###Reconciliation_Status <-Set up check box filter
###Cost_Recording_Status <-Set up check box filter
###Order_Bucket>"Complete Bucket" <-Set up check box filter
###Order_Bucket>"Pending Bucket" <-Set up check box filter

    
    BrandFilterSD = GroupedSD[(GroupedSD['attribute_set']==SB)]
    Year  = np.array(BrandFilterSD["OD_Year"]).reshape((-1, 1))
    Sales = np.array(BrandFilterSD["count"])    
    
#Need to make exception that if little data, error produced or just the tabular data outputed
    BrandData = FilterSD[(FilterSD['attribute_set']==SB)]
                  
                  
    MarginData =(BrandData.groupby(["attribute_set","OD_Year"], as_index=False)
                 .agg({"Net_Retail_Price_CAD_":["sum"], 
                       "Total_Cost_CAD_":["sum"]
                      }))
    MarginData.columns = ["".join(x) for x in MarginData.columns.ravel()]
    
    MarginData = MarginData.assign(TotalMargin = lambda x: 
                 MarginData["Net_Retail_Price_CAD_sum"] - MarginData["Total_Cost_CAD_sum"])

    MarginData = MarginData.assign(PercentMargin = lambda x: 
                 (MarginData["TotalMargin"] / MarginData["Net_Retail_Price_CAD_sum"])*100) 
    
    plt.plot(BrandFilterSD["OD_Year"],BrandFilterSD["count"])
    plt.plot(MarginData["OD_Year"],MarginData["PercentMargin"])
    plt.show()

    LinearModel = LinearRegression().fit(Year,Sales)
    Slope = LinearModel.coef_
    RSqr= LinearModel.score(Year,Sales)
    
    if Slope > 0: 
        SlopeDirection = "Positive"
    else: 
        SlopeDirection = "Negative"
    
    print("The Slope of "+ SB+ " is "+SlopeDirection)
    
    
    

In [109]:
#Create Input Widgets
SelectedBrand =  widgets.Dropdown(options=AllBrands,description='Brands:',disabled=False)


In [111]:
# %matplotlib inline
%matplotlib widget
interact(BrandPlotter, SB = SelectedBrand)

interactive(children=(Dropdown(description='Brands:', options=('Volant', 'Airaid', 'K&N', 'MBRP', 'AEM', 'Bull…

<function __main__.BrandPlotter(SB)>

## DEVELOPMENT

In [99]:
#Group by Attribute set & year sold, then count number of sales
GroupedSD = (FilterSD.groupby(["attribute_set","OD_Year"], as_index=False)
                ['Order_Date'].
                agg({"count":"count"}).sort_values(["attribute_set","OD_Year"] , ascending = True))



#Create the aggregateElement Class
class AggE:
  def __init__(self, ColName, fun, NewColName):
    self.ColName = str(ColName)
    self.fun = str(fun)
    self.NewColName = str(NewColName)
  def agger(self):
    return ("'" +self.ColName+ "':['"+ self.fun + "']" )



OrderDateCount = AggE("Order_Date","count","TotalSales")
NRetailPriceSum = AggE("Net_Retail_Price_CAD_","sum","Total_Retail_Price")o

NRetailPriceSum.agger()




aggregate = [OrderDateCount, NRetailPriceSum]

for g in aggregate:
    ColName = g.ColName
    fun = g.fun
    NewColName = g.NewColName
    
Order = ("'" +ColName+ "':['"+ fun + "']" )

Order = (ColName fun:[] + "']" )



def DataGrouper(df, group, aggregate):
    Output = (df
    .groupby(group, as_index=False)
    .agg(aggregate))
    
    Output.columns = ["_".join(x) for x in Output.columns.ravel()]
    
    return Output

DataGrouper(df = GlobalFilter, group = ["OD_Year"], aggregate = {Order)


{"A":"B":"C"}

SyntaxError: invalid syntax (<ipython-input-99-fdaa6b3f2154>, line 20)

In [53]:
#Compiled Output functions
#Plot Brands sales per year 
def BrandPlotter(SB):
    
    #Global Function Filter
    GlobalFilter =    SD[(SD['Order_Date']!="")
                   & (SD['attribute_set']==SB)
                   ]
    
    #Filtered Data Meta Data
    RecentSale = GlobalFilter.Order_Date.max()
    OldestSale = GlobalFilter.Order_Date.min()

#Setup Interactive Global Filter
###Reconciliation_Status <-Set up check box filter
###Cost_Recording_Status <-Set up check box filter
###Order_Bucket>"Complete Bucket" <-Set up check box filter
###Order_Bucket>"Pending Bucket" <-Set up check box filter

    #Data grouping
    ##Group by Attribute set & year sold, then count number of sales
    GroupedSD = (GlobalFilter
                 .groupby(["OD_Year"], as_index=False)
                ['Order_Date'].
                agg({"count":"count"}).sort_values(["OD_Year"] , ascending = True))
    
    MarginData = (GlobalFilter
                 .groupby(["attribute_set","OD_Year"], as_index=False)
                 .agg({"Net_Retail_Price_CAD_":["sum"], 
                       "Total_Cost_CAD_":["sum"]

    
    BrandFilterSD = GroupedSD[(GroupedSD['attribute_set']==SB)]
    Year = np.array(BrandFilterSD["OD_Year"]).reshape((-1, 1))
    Sales = np.array(BrandFilterSD["count"])    
                      }))
    
    MarginData.columns = ["".join(x) for x in MarginData.columns.ravel()]
    
    MarginData = MarginData.assign(TotalMargin = lambda x: 
                 MarginData["Net_Retail_Price_CAD_sum"] - MarginData["Total_Cost_CAD_sum"])

    MarginData = MarginData.assign(PercentMargin = lambda x: 
                 (MarginData["TotalMargin"] / MarginData["Net_Retail_Price_CAD_sum"])*100)
                                                          
    fig = plt.figure()
    fig.subplots_adjust(hspace=1, wspace=0.4) #Plot Spacer
    ax1 = fig.add_axes([0.1, 0.5, 1, 1.5],
                   xticklabels=[], ylim=(Sales.min(), Sales.max()))
    ax2 = fig.add_axes([0.1, 0.1, 1, .6],
                   ylim=(MarginData["PercentMargin"].min(), MarginData["PercentMargin"].max()))

    ax1.plot(BrandFilterSD["OD_Year"],BrandFilterSD["count"])
    ax2.plot(MarginData["OD_Year"],MarginData["PercentMargin"])
    plt.show()

    LinearModel = LinearRegression().fit(Year,Sales)
    Slope = LinearModel.coef_
    RSqr= LinearModel.score(Year,Sales)
    
    if Slope > 0: 
        SlopeDirection = "Positive"
    else: 
        SlopeDirection = "Negative"
    
    print("The Slope of " + SB + " is " + SlopeDirection)

    
#TroubleShoot
if False:
    SB = "Accel"
    BrandFilterSD = GroupedSD[(GroupedSD['attribute_set']==SB)]
    Year  = np.array(BrandFilterSD["OD_Year"]).reshape((-1, 1))
    Sales = np.array(BrandFilterSD["count"])    
    
#Need to make exception that if little data, error produced or just the tabular data outputed
    BrandData = FilterSD[(FilterSD['attribute_set']==SB)]
                  
                  
    MarginData =(BrandData.groupby(["attribute_set","OD_Year"], as_index=False)
                 .agg({"Net_Retail_Price_CAD_":["sum"], 
                       "Total_Cost_CAD_":["sum"]
                      }))
    MarginData.columns = ["".join(x) for x in MarginData.columns.ravel()]
    
    MarginData = MarginData.assign(TotalMargin = lambda x: 
                 MarginData["Net_Retail_Price_CAD_sum"] - MarginData["Total_Cost_CAD_sum"])

    MarginData = MarginData.assign(PercentMargin = lambda x: 
                 (MarginData["TotalMargin"] / MarginData["Net_Retail_Price_CAD_sum"])*100) 
    
    plt.plot(BrandFilterSD["OD_Year"],BrandFilterSD["count"])
    plt.plot(MarginData["OD_Year"],MarginData["PercentMargin"])
    plt.show()

    LinearModel = LinearRegression().fit(Year,Sales)
    Slope = LinearModel.coef_
    RSqr= LinearModel.score(Year,Sales)
    
    if Slope > 0: 
        SlopeDirection = "Positive"
    else: 
        SlopeDirection = "Negative"
    
    print("The Slope of "+ SB+ " is "+SlopeDirection)
    
    
###Net_Retail_Price_CAD_
###Total_Cost_CAD_
    
    #BrandPlotter("Accel")

SyntaxError: invalid syntax (<ipython-input-53-cf77f4cf0855>, line 33)

## TROUBLESHOOTING

In [98]:
((((2.5*365)/24)/30)*(80-29))/12
2.5*7

17.5

In [97]:
20*6*4

480

In [95]:
3*365*(80-29)

55845

In [96]:
def f(x):
    return x

interact(f, x=10);


interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…